# Test incident reports
## [TIR1](#TIR1)
### Description
When VMOTOR turns on, there's parasitic power that feeds back to the gate of Q305, turning VMOTOR off.

### Cause
The main issue is that USB data lines are feeding into VUSB through D401.  Insert a schottky diode between VUSB and D401 to prevent this.

### Solution
Remove R117 because it makes a path from +3V3 to R115, Q100 into VDBG.  Removing R117 still allows the MCU to be programmed.

## [TIR2](#TIR2)
### Description
Sleep current is too high

### Cause
* A parasitic current flows from VBAT to body diode of Q300, to R304, R303 through D106, D107, D108 , R110, R111, R112 to 3V3.  The leakage of D106, D107 and D108 causes the leakage.  To make matters worse, these diodes are in parallel, so their leakage currents add up.
* Because the BAV170 was not available, BAT54C diodes have been used.  BAT54C has a much higher leakage current.

### Solution
* D106, D107 and D108 can be replaced by BC856B (emitter to pin 1 of BAT54, collector to pin 2 of BAT54, base to pin 3 of BAT54).  The power supply current in sleep mode is 0.6µA.

## [TIR3](#TIR3)
### Description
The motor turns during programming

### Cause
* When the IO6 pin of the MCU is HiZ, it's not low.  
* This voltage level on IO6 is odd, because the motor controller has internal pull down resistors.
* The voltage on the input of the motor controller must be <0.5V to keep it off.

### Solution
* Adding a 10k pull down resistor, pulls the voltage on IO6 to 0.66V, which is turns the motor off in our case, but it's not a guarantee.
* A 3k3 pull down resistor (a value already present in the design) pulls the voltage on IO6 to 0.235V, which is low enough to keep the motor off.

## [TIR4](#TIR4)
### Description
The motor doesn't turn when powered by USB

### Cause
* The rectifier diode D301 has a voltage drop of 0.6V.  The resulting voltage is too low for the stepper motor driver to work.

### Solution
* A motor driver with a lower forward voltage is not really an option.  We would have to use a motor driver with external sensing circuitry.  Moreover, this would be a single-source component.
* The solution is to use a schottky diode with a lower forward voltage.  In this case, there is no need to select a low-leakage diode.  When the system is sleeping, VMOTOR will be low and there will be no leakage current through the diode when VUSB and VDBG are not present.

## [TIR5](#TIR5)
### Description
The voltage on the I2C bus is too high.  

### Cause
There's an error in the I2C bus going to the LCD.  The gates of the NMOS are pulled up to 5V.  The NMOS transistors have a low Vg threshold.  The source can never rise high enough to turn the NMOS off.  So the NMOS is always conducting.

### Solution
* The NMOS gates should be connected to 3V3 instead of 5V.

## [TIR6](#TIR6)
### Description
The device can no longer be powered by VBAT

### Cause
* The channel resistance of Q303 in the off-state reduced to 1.4kOhm, Preventing MOTOR_GATE from going low enough to turn Q300 and Q301 on.  The cause is uncertain, but it might have been current surges when discharging gates of Q300 and Q301.  
* Later on, it has been found that a non-earthed soldering iron had been used.  The tip of the iron is at >100VAC, while the PCB being soldered is connected to earth by the oscilloscope probes.
The gate of the SI3401A has no zener protection, so it's likely that the gate oxide has been damaged.

### Solution
* Replacing this part of the circuitry and using a PNP-transistor (BC856B) instead of a PMOS (AO3401A) should solve the problem.
* Power the Pinecil soldering iron by an earthed power supply.

## [TIR7](#TIR7)
### Description
Voltage drops on VMOTOR when powered by USB

### Cause
The voltage drop on USB causes the input voltage to drop below the minimum voltage of the motor driver.  

### Solution
* Adding extra capacitance to the USB power supply is not feasible because according to simulations, it would require > 10000µF to get rid of the drop (5ohm power supply, 1ohm internal resistance, 200mA current peak during 2ms).
* Add a schottky diode between VMOTOR and the capacitors feeding the motor driver.

## [TIR8](#TIR8)
### Description
Voltage switchover from USB to battery is not working.  There's a voltage drop on VMOTOR, causing the device to reset

### Cause
* The USB voltage drops rapidly down to 2.27V. The voltage on the gate of Q305 follows that voltage drop well.  It's one diode drop below VUSB.
* When VUSB drops below 2.27V, the load switches off and VUSB drops much more slowly.  It takes 29ms for the gate of Q305 to drop below 0.78V, at which point Q305 turns off.
* The problem is that Q305 switches off too late.  It should switch off well before VUSB drops below 2.27V.

### Solution
* Change C408 from 10µF to 100nF so that VUSB drops faster.
* Add 10µF to C303 so that VMOTOR drops slower.
* Add 100µF elco on VMOTOR
* Add a 3.3K resistor between cathode of D302 and gate of Q305
* Replace R305 by 1K

# Power test

## Power enable by press of a momentary switch
* **Test:** Power on by switch
* **Expected result:**T Power on when switch pushed
* **Actual result:**T power turns on and off when switch pushed
* **Status:** ❌ [TIR1](#TIR1)

## Powered by battery
### Reverse polarity
* **Test:** Connect 6V lab power supply with reverse polarity
* **Expected result:** No damage, Vmotor remains at 0V
* **Actual result:** 
  * Vmotor remains at 1.2mV
  * When applying correct polarity, Vmotor is 5.99V and the MCU works again.
* **Status:** ✅

### Voltage drop over Q300 and Q301
* **Test:** 
  * Connect a 6V lab power supply to the battery terminals.
  * Disable 5V by pulling U301.4 low.
  * Apply a 500mA constant current load to VMOTOR.
  * Measure the voltage at the battery terminals and at the VMOTOR net.
* **Expected result:** The voltage at the battery terminals xxx mV higher than at the VMOTOR net.
  * rdsON is 45mohm for Vgs = 4.5V -> Voltage drop over Q300 and Q301 is 22.5mV at 500mA. -> 45mV total.
* **Actual result:** 
  * VBAT = 5.95V
  * VMOTOR = 5.89V
  * VBAT - VMOTOR = 60mV
  * Vgs(Q300) = 5.89 - 0.69 = 5.2V
* **Status:** 45mV is close enough to 60mV ✅

## Powered by debug port
* Power the board with the SKEDD connector only.

### VMOTOR voltage
* **Test:** Measure voltage on VMOTOR
* **Expected result:** VMOTOR is a diode drop lower than VDBG.
* **Actual result :** 
  * VDBG = 4.55V
  * VMOTOR = 4.05V
* **Status:** ✅
  * D300 has a 500mV voltage drop, which is significant when compared to a common schottky diode.  We trade off forward voltage with reverse leakage.

### Battery short circuit
* **Test:** short circuit the battery terminals
* **Expected result:** VMOTOR voltage should not change (i.e. remain a diode drop lower than VDBG)
* **Actual result:** 4.02V
* **Status:** ✅

### Motor driving test with D300 is B340AF
This is a test in an effort to fix [TIR4](#TIR4).
* **Test:** Run the motor
* **Expected result:** Motor runs
* **Actual result:**
  Motor is running.
  * VDBG = 4.92V
  * VMOTOR = 4.66V
* **Status:** ✅

## Powered by USB port
### Power integrity
* **Test:** Power the board a 50cm lon USB-C cable.  Check for voltage drop.
* **Expected result:** Voltage drop < 100mV
* **Actual result:** 200mV voltage drop during 2ms when WiFi is enabled.
* **Status:** ❌ [TIR7](#TIR7)

### DC-motor test
* **Test:** Run the motor
* **Expected result:** Motor runs
* **Actual result:** Motor doesn't run
* **Status:** ❌ [TIR4](#TIR4)

### Power switchover
* **Test:** Power the board with USB and battery connected.  Disconnect USB.
* **Expected result:** Board continues to run on battery.
* **Actual result:** Board reset because there's a 30ms gap in power.
* **Status:** ❌ [TIR8](#TIR8)

## VRTC
* **Test:** Measure VRTC voltage when only battery is connected and MCU is not powered.
* **Expected result:** VRTC is between 1.8Vmin, 2.2Vtyp and 2.9Vmax
* **Actual result:** 2.11V
* **Status:** ✅

## Sleep current
* **Test:** Measure sleep current when only battery is connected and MCU is not powered and RTC is not outputting a square wave.
* **Expected result:** Sleep current around 500nA
* **Actual result:** 1.3µA
* **Status:** ❌ [TIR2](#TIR2)

## Communication test
### USB VID:PID
* **Test:** Get USB VID:PID
* **Expected result:** ❓
* **Actual result:** 303A:0001 Espressif USB JTAG/serial debug unit
* **Status:** ✅

### USB serial
* **Test:**
  * Connect to USB serial port and send and receive data
  * Enable USB-CDC in platformio.ini with '-DARDUINO_USB_MODE=1'
* **Expected result:** Receive data and send data works
* **Actual result:** Received data and sent data works
* **Status:** ✅

### I2C
* **Test:** Measure voltage on SDA and SCL
* **Expected result:** SDA and SCL are pulled up to 3.3V
* **Actual result:** SDA and SCL are pulled up to 4V
* **Status:** ❌ [TIR5](#TIR5)

## Peripheral test
### Voltage monitor for VMOTOR
* **Test:** measure VMOTOR with ADC of the MCU
* **Expected result :** within 1% of VMOTOR
* **Actual result:**
  * ADC = 4048mV 
  * DMM = 4.02V
* **Status:** ✅

### Motor
#### Motor VREF
* **Test:** Measure VREF of the motor driver
* **Expected result:** 2.25V
* **Actual result:** 2.24V
* **Status:** ✅

#### Motor inputs
* **Test:** Connect motor inputs to MCU.  When MCU is powered and being programmed, the motor inputs should be high impedance.  Motor should not turn.
* **Expected result:** Motor does not turn
* **Actual result:** Motor turns
* **Status:** ❌ [TIR3](#TIR3)

### RTC
#### RTC frequency
* **Test:** Measure RTC frequency
* **Expected result:** 32.768kHz +/- 20ppm (based on 20ppm crystal)
* **Actual result:**
  * Using 15pF : 32.76822kHz  <= Using 15pF
  * Using 18pF : 32.76745kHz
* **Status:** ✅

#### Crystal drive level 
* **Test:** Measure peak-peak voltage on RTC crystal and calculate drive level
* **Expected result:** ?, but should be within 1µW
* **Actual result:** Vpp = 340mV
* **Status:** ✅

In [ ]:
import math
Vpp = 0.34 # [V]
f = 32.768e3 # [Hz]
Cl = 21e-12 # [F], approximated (15pF * 2/2 + 6pF)
ESR = 70e3 # [Ohm]

P = (Vpp * math.pi * f * Cl)**2 * ESR/2 
print("Power dissipation: {:.2f} µW".format(P*1e6))

Power dissipation: 0.02 µW


### Momentary switches
#### SW100
* **Test:** Press SW100 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is 0.6V
* **Actual result:** 0.63V
* **Status:** ✅

#### SW101
* **Test:** Press SW101 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is  2.7V/2  + 0.6 = 1.95V
* **Actual result:** 1.96V
* **Status:** ✅

#### SW102
* **Test:** Press SW102 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is 6.6/9.9 * 2.7V  + 0.6 = 2.4V
* **Actual result:** 2.4V
* **Status:** ✅

The voltage divider consists of 3x3K3 resistors.  The difference between pressing SW100 and SW101 is 1.35V, while the difference between SW101 and SW102 is only 0.45V.  This is a ratio of 3:1, which is undesirable. The ratio should be closer to 1:1 to have more noise margin in the ADC.

In [ ]:
def voltage_divider(R1, R2, Vin):
    return Vin * R2 / (R1 + R2)

# Suggestion for improvement: getting more diffential voltage between the button presses, using resistor values already on the board.
R110 = 10e3 # [Ohm]
R111 = 5.1e3 # [Ohm]
R112 = 15e3 # [Ohm]

Voffset = 0.6 # [V]
Vr = 3.3 - Voffset # [V]

# SW100 pressed
Vout1 = Voffset # [V]

# SW101 pressed
Vout2 = voltage_divider(R110, R111, Vr) + Voffset
print("SW101: Vout: {:.2f} V".format(Vout2))

# SW102 pressed
Vout3 = voltage_divider(R110, R111 + R112, Vr) + Voffset
print("SW102: Vout: {:.2f} V".format(Vout3))

# Nothing pressed
Vout4 = 3.3 # [V]

# Ratio (SW100-SW101) / (SW101-SW102)
ratio = (Vout1 - Vout2) / (Vout2 - Vout3)
print("Ratio: {:.2f}".format(ratio))

SW101: Vout: 1.51 V
SW102: Vout: 2.40 V
Ratio: 1.02


### Momentary switches after modification
R110 = 10K, R111 = 5.1K, R112 = 15K

#### SW100
* **Test:** Press SW100 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is 0.6V
* **Actual result:** 0.599V
* **Status:** ✅

#### SW101
* **Test:** Press SW101 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is  1.51V
* **Actual result:** 1.505V
* **Status:** ✅

#### SW102
* **Test:** Press SW102 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is 2.4V
* **Actual result:** 2.40V